## Discover UDP socket

In this excercice we will use RIOT UDP sock API. This API provides a set of functions to establish connections or send and receive datagrams using UDP. For this purpose we implement two UDP applications using the Posix compliant API: one client will periodically send the "Hello Server" message, the server prints any messages received.

The server listens on port 8888. The client sends its messages to the IPv6 link-local address of the server.

### Implement server application

1. Go into the server application directory
2. Add the posix sockets module dependency to the Makefile

```mk
USEMODULE += posix_sockets
```

3. In the main.c file, create the server socket

```c
    int server_socket = socket(AF_INET6, SOCK_DGRAM, IPPROTO_UDP);
    if (server_socket < 0) {
        puts("error initializing socket");
        server_socket = 0;
        return 1;
    }
```

4. Instanciate and configure the server address

```c
    struct sockaddr_in6 server_addr;
    server_addr.sin6_family = AF_INET6;
    memset(&server_addr.sin6_addr, 0, sizeof(server_addr.sin6_addr));
    server_addr.sin6_port = htons(SERVER_PORT);
```

5. Bind the socket to the server address

```c
   if (bind(server_socket, (struct sockaddr *)&server_addr,
            sizeof(server_addr)) < 0) {
    server_socket = -1;
    puts("error binding socket");
    return 1;
    }
```

6. Start listening to incoming packets

```c
    int res;
    struct sockaddr_in6 src;
    socklen_t src_len = sizeof(struct sockaddr_in6);
    if ((res = recvfrom(server_socket, server_buffer, sizeof(server_buffer), 0,
                       (struct sockaddr *)&src, &src_len)) < 0) {
        puts("Error on receive");
    }
    else if (res == 0) {
        puts("Peer did shut down");
    }
    else {
        printf("Message received: ");
        puts(server_buffer);
        sendto(server_socket, SERVER_MESSAGE, sizeof(SERVER_MESSAGE), 0,
               (struct sockaddr *)&src, sizeof(src));
    }
```

Compile and verify your application code with native target is then as simple as running the following command

In [ ]:
!make -C server

### Implement client application

1. Go into the client application directory
2. Add the posix sockets module dependency to the Makefile

```mk
USEMODULE += posix_sockets
```

3. In the main.c file, create the socket

```c
    int s = socket(AF_INET6, SOCK_DGRAM, IPPROTO_UDP);
    if (s < 0) {
        puts("error initializing socket");
        return 1;
    }
```

4. Configure the destination address (based on SERVER_ADDR and SERVER_PORT macros)

```c
    /* configure the destination address */
    struct sockaddr_in6 dst;
    dst.sin6_family = AF_INET6;

    /* parse destination address */
    ipv6_addr_from_str((ipv6_addr_t *)&dst.sin6_addr, SERVER_ADDR);

    /* parse port */
    dst.sin6_port = htons(SERVER_PORT);
```

5. Sends packets to the server

```c
   sendto(s, CLIENT_MESSAGE, strlen(CLIENT_MESSAGE), 0,
             (struct sockaddr *)&dst, sizeof(dst));
```


Compile and verify your application code with native target and the following command

In [ ]:
!make -C client

### Launch an experiment

Choose your site (grenoble|lille|strasbourg):

In [ ]:
%env SITE=grenoble

1. Submit an experiment with two nodes

In [ ]:
!iotlab-experiment submit -d 60 -l 2,archi=m3:at86rf231+site=$SITE

2. Wait for the experiment to be in the Running state:

In [ ]:
!iotlab-experiment wait

3. Check the resources allocated to the experiment

In [ ]:
!iotlab-experiment get -ni

From here you will choose one node with the server role and the other one with client role. You need to open a terminal for the server

4. For server node, open a Jupyter terminal (use `File > New > Terminal`) and run the command replacing `<id> <site>`with the right values

Here the link-local address of the server is **fe80::f09d:f4ff:fe58:14d4**. Keep the terminal open and the server application running.

5. For client node, open a Jupyter terminal (use `File > New > Terminal`) and run the command replacing `<link-local-addr> <id> <site>' with the right value

On the server terminal, you should see the following messages:

```mk
Message received: Hello Server
Message received: Hello Server
Message received: Hello Server
Message received: Hello Server
Message received: Hello Server
```